In [1]:
import cv2 as cv
import numpy as np
import tensorflow as tf
import imutils
from imutils.contours import sort_contours

In [2]:
chars=[]

In [3]:
type(chars)

list

In [4]:
labelNames = ['!', '"', '#', '$', '%', '&', "’", '(', ')', '*', '+', ',', '-',
       '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':',
       ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
       'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
       'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a',
       'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{',
       '|', '{', '~']

In [7]:
source="C:/Users/shubh/Downloads/HELLO_FINAL.jpeg"
img=cv.imread(source)
gray=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
blur=cv.GaussianBlur(gray, (11,11), cv.BORDER_DEFAULT)
canny=cv.Canny(img, 100, 300)

In [8]:
cont = cv.findContours(canny.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
cont = imutils.grab_contours(cont)
cont = sort_contours(cont, method="left-to-right")[0]

In [10]:
model=tf.keras.models.load_model("C:/Users/shubh/Desktop/Btech/MINI PROJECT/OCR/OCR local file/OCR_MODEL_edited.h5")

In [11]:
from numpy import asarray
for i in cont:
    (x, y, w, h) = cv.boundingRect(i)
    img = cv.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    if (w >= 10 and w <= 150) and (h >= 20 and h <= 120):
        ROI = gray[y:y + h, x:x + w]               #cropping
        thresh = cv.threshold(ROI, 0, 255, cv.THRESH_BINARY_INV, cv.THRESH_OTSU)[1]
        (tH, tW) = thresh.shape
        dX = int(max(0, 32 - tW) / 2.0)
        dY = int(max(0, 32 - tH) / 2.0)
        padded = cv.copyMakeBorder(thresh, top=dY, bottom=dY, left=dX, right=dX, borderType=cv.BORDER_CONSTANT, value=(0, 0, 0))
        padded = cv.resize(padded, (28, 28))
        """ cv.imshow("image", padded)
        cv.waitKey(0) """
        padded = asarray(padded)
        padded = np.expand_dims(padded, axis=-1)
        chars.append((padded, (x, y, w, h)))
boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")
preds = model.predict(chars)


1/1 [==============================] - 0s 316ms/step


In [12]:
output = ""
for (pred, (x, y, w, h)) in zip(preds, boxes):
    i = np.argmax(pred)
    prob = pred[i]
    label = labelNames[i]
    output += label

print("output",output)
print(len(output))

output HELLN
5


In [ ]:
""" CORRECT OUTPUT - "HELLO"
PREDICTED OUTPUT - "HELLN" """